In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
from glob import glob
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [3]:
pip install pickle5

In [4]:
import pickle5 as pickle

In [5]:
from glob import glob
import pickle5 as pickle
import numpy as np

ROOT_PATH = "../input/"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    if split=="train":
        f_in = ROOT_PATH + split + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        inputs = np.asarray(inputs)

        outputs = None
    
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)
        
    elif split=="test":
        f_in = ROOT_PATH + "test-inputs" + "/" + city + "_inputs"
        inputs = pickle.load(open(f_in, "rb"))
        inputs = np.asarray(inputs)

        outputs = None
        
    return inputs, outputs

# class ArgoverseDataset(Dataset):
#     """Dataset class for Argoverse"""
#     def __init__(self, city: str, split:str, transform=None):
#         super(ArgoverseDataset, self).__init__()
#         self.transform = transform

#         self.inputs, self.outputs, self.dit_int, self.dit_out = get_city_trajectories(city=city, split=split, normalized=False)

#     def __len__(self):
#         return len(self.inputs)

#     def __getitem__(self, idx):

#         data = (self.inputs[idx], self.outputs[idx])
            
#         if self.transform:
#             data = self.transform(data)

#         return data

# # intialize a dataset example
# city = 'palo-alto' 
# split = 'train'
# train_dataset  = ArgoverseDataset(city = city, split = split)

In [6]:
class EncoderRNN(nn.Module):
    """Encoder Network."""
    def __init__(self,
                 input_size: int = 2,
                 embedding_size: int = 8,
                 hidden_size: int = 16):
        """Initialize the encoder network.
        Args:
            input_size: number of features in the input
            embedding_size: Embedding size
            hidden_size: Hidden size of LSTM
        """
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(input_size, embedding_size)
        self.lstm1 = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x: torch.FloatTensor, hidden):
        """Run forward propagation.
        Args:
            x: input to the network
            hidden: initial hidden state
        Returns:
            hidden: final hidden 
        """
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm1(embedded, hidden)
        return hidden
    

In [7]:
class DecoderRNN(nn.Module):
    """Decoder Network."""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        """Initialize the decoder network.
        Args:
            embedding_size: Embedding size
            hidden_size: Hidden size of LSTM
            output_size: number of features in the output
        """
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm1 = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        """Run forward propagation.
        Args:
            x: input to the network
            hidden: initial hidden state
        Returns:
            output: output from lstm
            hidden: final hidden state
        """
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm1(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden

In [9]:
# convert the original training document into trainable batches 
def get_batches(inp, out, batch_size):
    
    num_batches = int(len(inp) / (batch_size))
    
    X = inp[:num_batches*batch_size]
    Y = out[:num_batches*batch_size]
#     Y = np.zeros((len(X), 50))
    
    
    # generate trainable batches
    for i in range(0, num_batches*batch_size, batch_size):
        Xs = []
        Ys = []
        for j in range(i, i+batch_size):
            temp_x = X[j]
               
            temp_y = Y[j]
            
            Xs.append(temp_x)
            Ys.append(temp_y)
        
        Xs = np.array(Xs)
        Ys = np.array(Ys)

        yield torch.tensor(Xs).float(), torch.tensor(Ys).float()

In [15]:
def train_model(inp, out, n_epochs, hidden_size, batch_size, seq_size, lr, temperature):
    
    encoder = EncoderRNN(2, 8, hidden_size)
    decoder = DecoderRNN(8, hidden_size, 2)
    
    encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=lr)
    decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
    criterion = nn.MSELoss()
    train_loss_over_time = []
    epochs = []
    for epoch in range(n_epochs):
        
        batches = get_batches(inp, out, batch_size)
         
        j = 0
        print("new epoch starts, currently at " + str(epoch) + " epoch")

        
        for X, Y in batches:
            encoder.train()
            decoder.train()

            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            
            encoder_hidden = (torch.zeros(batch_size, hidden_size), torch.zeros(batch_size, hidden_size))
    
            loss = 0
                
            temp_x = X
#             print(temp_x.shape)
            temp_y = Y
    
            for i in range(len(temp_x[0])):
                encoder_input = temp_x[:, i, :]
                encoder_hidden = encoder(encoder_input, encoder_hidden)
            
            decoder_input = encoder_input[:, :2]
            decoder_hidden = encoder_hidden
            
            
            for i in range(60):
                decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
                
#                 decoder_outputs[:, i, :] = decoder_output

                # Update loss
                loss += criterion(decoder_output[:, :2], temp_y[:, i, :2])

                decoder_input = decoder_output
 

            loss = loss / 60
            
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
            
            j += 1
        
        print(decoder_output[:, :2][0])
        print(temp_y[:, 59, :2][0])    
        print("loss at last batch " + str(j) + " - " + str(loss))
        print("\n")  
        
        train_loss_over_time.append(loss.item())
        epochs.append(epoch)
    
    
    plt.plot(epochs, train_loss_over_time)
    plt.xlabel("epoch num")
    plt.ylabel("train loss")
    return encoder, decoder

In [12]:
p_i, p_o = get_city_trajectories(city="palo-alto", split="train", normalized=False)

In [13]:
import time

In [16]:
# (inp, n_epochs, hidden_size, batch_size, seq_size, lr, temperature)
start = time.time()
print("Starts training process")
encoder, decoder = train_model(p_i, p_o, 20, 128, 500, 110, 0.05, 1.5)
end = time.time()
print("Ends training process, total time - " + str(end - start))